In [45]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

import time
import scipy
from PIL import Image
from scipy import ndimage
import skimage
from dnn_utils_v2 import *


%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
#ACTIVIDAD 1: Cree e inicialice los parámetros de la red neuronal de 2 capas. 
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.random.randn(n_h, 1)*0.01
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.random.randn(n_y, 1)*0.01
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    parameters = {"W1": W1,
    "b1": b1,
    "W2": W2,
    "b2": b2}
    
    return parameters
#probando
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[ 0.01744812]
 [-0.00761207]]
W2 = [[ 0.00319039 -0.0024937 ]]
b2 = [[0.01462108]]


In [47]:
#ACTIVIDAD 2: Implementar la inicialización de una red neuronal de capa L
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    # numero de capas en la neural
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1)) 
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters
#probando
parameters = initialize_parameters_deep([5,4,3])

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [48]:
#ACTIVIDAD 3: Implementar la propagación hacia adelante lineal. 
# FUNCTION: linear_forward
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    return Z, cache
#Probando la función
A, W, b = linear_forward_test_case()
Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [49]:
#actividad 4: Implementar  la propagación hacia adelante de la capa LINEAR-> ACTIVACIÓN
# FUNCTION: linear_activation_forward
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    return A, cache

In [50]:
#Probando a la función
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation="sigmoid")
print("With sigmoid: A = " + str(A))
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [51]:
#actividad 5  Implementar la propagación hacia adelante del modelo anterior
# FUNCTION: L_model_forward
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    caches = list()
    # número de capas de la red
    # Implemente [LINEAR -> RELU]*(L-1). Agregue "cache" a la lista "caches"
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, 
                                             parameters["W" + str(l)], 
                                             parameters["b" + str(l)], 
                                             activation='relu')
        caches.append(cache)
    
    # Implementa LINEAR -> SIGMOID. Agregue "cache" a la lista "caches".
    
    AL, cache = linear_activation_forward(A, 
                                             parameters["W" + str(L)], 
                                             parameters["b" + str(L)], 
                                             activation='sigmoid')
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
    return AL, caches

In [52]:
#Probando la función
X, parameters = L_model_forward_test_case_2hidden()
print(parameters)
print(len(parameters)//2)
print("W"+str(2))
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

{'W1': array([[ 0.35480861,  1.81259031, -1.3564758 , -0.46363197,  0.82465384],
       [-1.17643148,  1.56448966,  0.71270509, -0.1810066 ,  0.53419953],
       [-0.58661296, -1.48185327,  0.85724762,  0.94309899,  0.11444143],
       [-0.02195668, -2.12714455, -0.83440747, -0.46550831,  0.23371059]]), 'b1': array([[ 1.38503523],
       [-0.51962709],
       [-0.78015214],
       [ 0.95560959]]), 'W2': array([[-0.12673638, -1.36861282,  1.21848065, -0.85750144],
       [-0.56147088, -1.0335199 ,  0.35877096,  1.07368134],
       [-0.37550472,  0.39636757, -0.47144628,  2.33660781]]), 'b2': array([[ 1.50278553],
       [-0.59545972],
       [ 0.52834106]]), 'W3': array([[ 0.9398248 ,  0.42628539, -0.75815703]]), 'b3': array([[-0.16236698]])}
3
W2
AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


In [53]:
#ACTIVIDAD 6: Calcule el costo de entropía cruzada J¿: 
def compute_cost(AL, Y):
    m=Y.shape[1]
    #cost = (np.sum(np.dot(Y,np.log(AL)-T)+ (1-Y)*(np.log(1-AL))))/(-m) 
    cost = ( np.sum( np.multiply(Y,np.log(AL) ) + np.multiply( (1-Y),( np.log(1-AL) ) ) ) )/(-m) 
    
    cost = np.squeeze(cost)
    assert( cost.shape == ())
    return cost

#probando la funcion
Y, AL = compute_cost_test_case()

print("cost="+str(compute_cost(AL, Y)) )

cost=0.414931599615397


In [54]:
#ACTIVIDAD 7: Usa las 3 fórmulas anteriores para implementar linear_backward (). 
def linear_backward(dZ, cache):  
    A_prev, W, b = cache 
    m = A_prev.shape[1]
    dW = (np.dot(dZ, A_prev.T))/m
    db = (np.sum(dZ,axis=1,keepdims = True))/m
    dA_prev = (np.dot(W.T, dZ))
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    return dA_prev, dW, db
#probando la funcion
dZ, linear_cache = linear_backward_test_case() 
dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW)) 
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


In [55]:
#actividad 8:Implementar la propagación hacia atrás para la capa LINEAR-> ACTIVACIÓN 
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache

    if activation == "relu":
        dZ =  relu_backward( dA, activation_cache )
        dA_prev, dW, db = linear_backward( dZ , linear_cache )

    elif activation == "sigmoid":
        dZ = sigmoid_backward ( dA, activation_cache )
        dA_prev, dW, db = linear_backward( dZ, linear_cache )

    return dA_prev, dW, db

#Probando la función
dAL, linear_activation_cache = linear_activation_backward_test_case()
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [56]:
#actividad 9:  Implementar la propagación hacia atrás para el modelo [LINEAR-> RELU] ×× (L-1) -> LINEAR -> SIGMOID. 
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # Iniciando el backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    # en cada capa l (SIGMOID -> LINEAR) gradients. Entrada: "dAL, current_cache". Salida: 
    # "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1] 
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward( dAL, current_cache, "sigmoid" )

    # Loop de l=L-2 to l=0
    for l in reversed(range(L-1)):
        # En cada capa l: (RELU -> LINEAR) gradients.
        # Entrada: "grads["dA" + str(l + 1)], current_cache". Salida: 
        #"grads["dA" + str(l)] , grads["dW" + str(l +1)] , grads["db" + str(l + 1)]

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)],current_cache,"relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads
#Probando la función
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)


dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [57]:
#actividad 10: implementar update_parameters()
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # numero de capas de la red
    for l in range(1,(L+1)):
        parameters["W" + str(l)] =(parameters["W" + str(l)] - learning_rate*(grads["dW" + str(l)]) )
        parameters["b" + str(l)] =(parameters["b" + str(l)] - learning_rate*(grads["db" + str(l)]) )
    return parameters

#Probando la función
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)
print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


In [58]:
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    print("hi5")
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [60]:
#train_x_orig, train_y, test_x_orig, test_y, classes = load_dataset()
train_x_orig, train_y, test_x_orig, test_y, classes = load_dataset()
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T  
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

capas = [12288,1] 

def prueba_deep_network(X, Y, dimesion, learning_rate = 0.0075, iteraciones = 2000, print_cost=False):#lr was 0.009
    np.random.seed(1)
    costs = []                         
    parameters = initialize_parameters_deep(capas)
    for i in range(0, iteraciones):
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate=learning_rate)
        if print_cost and i % 100 == 0:
            print ("Costo despues de iteracion ", str(i),": ", str(cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    return parameters
parameters = prueba_deep_network(train_x, train_y, dimesion, iteraciones = 1001, print_cost = True)


hi5
Costo despues de iteracion  0 :  0.6909085741648416
Costo despues de iteracion  100 :  1.4374833514095104
Costo despues de iteracion  200 :  0.8875469824759281
Costo despues de iteracion  300 :  0.6605040835244369
Costo despues de iteracion  400 :  0.48089711237327065
Costo despues de iteracion  500 :  0.3382697482227684
Costo despues de iteracion  600 :  0.23570580436785404
Costo despues de iteracion  700 :  0.19281195619529987
Costo despues de iteracion  800 :  0.17824368501194035
Costo despues de iteracion  900 :  0.16612673420336202
Costo despues de iteracion  1000 :  0.15558917256498056


In [65]:
layer = [12288,5,5,3,1] 
alpha = 0.01
num_iterations = 2001

def modelo_final(X, Y, dimesion, alpha = 0.01, num_iterations = 1800, print_cost=False):#lr was 0.009
    np.random.seed(1)
    costs = []                         
    parameters = initialize_parameters_deep(layer)
    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate=alpha)
        if print_cost and i % 100 == 0:
            print ("costo de ",str(i),": ",str(cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    return parameters

parameters = modelo_final(train_x, train_y, dimesion, alpha, num_iterations, print_cost = True)
print (parameters)

costo de  0 :  0.6931471837874629
costo de  100 :  0.6740962123836819
costo de  200 :  0.6625205543648794
costo de  300 :  0.6554527607019606
costo de  400 :  0.6511123116263571
costo de  500 :  0.6484315446710784
costo de  600 :  0.646767277866769
costo de  700 :  0.6457294487414328
costo de  800 :  0.6450798218914515
costo de  900 :  0.6446719174732634
costo de  1000 :  0.6444151348445231
costo de  1100 :  0.644253148254781
costo de  1200 :  0.6441507891262113
costo de  1300 :  0.6440860202221679
costo de  1400 :  0.6440449919394589
costo de  1500 :  0.6440189793786159
costo de  1600 :  0.6440024753482685
costo de  1700 :  0.6439919982049943
costo de  1800 :  0.6439853440557064
costo de  1900 :  0.6439811163991047
costo de  2000 :  0.6439784296153603
{'W1': array([[ 1.78862847e-02,  4.36509851e-03,  9.64974681e-04, ...,
         7.42032717e-03,  7.77721020e-03, -2.04410060e-02],
       [-2.03473729e-02, -1.27710592e-02, -8.45042725e-03, ...,
        -1.59285790e-02,  1.18975700e-02, 

In [61]:
#
      

NameError: name 'layers_dims' is not defined